In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.layers import Input, Dense, Layer, Multiply, Add, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


import tensorflow as tf
tf.config.run_functions_eagerly(True)

import tensorflow as tf
from bayes_opt import BayesianOptimization

# 1. Read Data
loadings_df = pd.read_csv('') #the path for the loadings file
micedata_df = pd.read_csv('') ##the path for the genomic data

# 2. Data Processing
micedata_features = micedata_df.iloc[:, 2:].values
loadings_real = loadings_df.applymap(lambda x: np.real(complex(x)) if isinstance(x, str) else np.real(x)).values

# 3. Feature Reduction using PCA
#here is for an example, 169 is not an experienced value, it is decided by the result from the loadings (please see the paper for the details). Please use the loadings code to get this correct value for your own data.
pca_micedata = PCA(n_components= 169)

loadings_real_reduced = pca_loadings.fit_transform(loadings_real)


# 4. Combine Features
combined_features = np.dot(micedata_features, loadings_real_reduced.T) # Changed to use micedata_features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(combined_features)
target = micedata_df.iloc[:, :2].values #for the two traits as example


# 5. Define the Layers
class GatedResidualUnit(Layer):
    def __init__(self, units, **kwargs):
        super(GatedResidualUnit, self).__init__(**kwargs)
        self.units = units
        self.dense = Dense(units, activation='elu', kernel_regularizer=l2(0.01))
        self.gate = Dense(units, activation='sigmoid', kernel_regularizer=l2(0.01))

    def call(self, inputs):
        residual = inputs
        x = self.dense(inputs)
        gate = self.gate(inputs)
        return Add()([Multiply()([x, gate]), residual])

class VariableSelectionNetwork(Layer):
    def __init__(self, units, **kwargs):
        super(VariableSelectionNetwork, self).__init__(**kwargs)
        self.units = units #the output size of the internal dense layer
        self.dense = Dense(units, activation='sigmoid', kernel_regularizer=l2(0.01))

      def call(self, inputs):
        # If input has fewer than `self.units`, it uses all available.
        if inputs.shape[1] < self.units:
            inputs_for_selection = inputs
        else:
            inputs_for_selection = inputs[:, :self.units] # Slice to the specified units

        dense_output = self.dense(inputs_for_selection)
        hard_sigmoid_output = tf.maximum(0.0, tf.minimum(1.0, 0.2 * dense_output + 0.5))
        return Multiply()([inputs_for_selection, hard_sigmoid_output])


#creat the grvsnn model
def create_grn_vsn_model(input_shape, dropout_rate=0.3, n_hidden_units = 128):
    inputs = Input(shape=input_shape)

    #step 1: GR block
    grn_output = GatedResidualUnit(units = n_hidden_units)(inputs) #here n_hidden_units is the dimensionality of GR block
    grn_output = Dropout(dropout_rate)(grn_output)

    #step 2: VS block
    selected_features = VariableSelectionNetwork(units = n_hidden_units)(grn_output) #here 128 is an example, which is the number of features the VS block will select from and process

    outputs = Dense(2)(selected_features)
    model = Model(inputs, outputs)
    return model

# 7. Bayesian Optimization for Hyperparameters
def train_model(lr, dropout_rate):
    lr = max(0.00001, min(lr, 0.001))
    dropout_rate = max(0.1, min(dropout_rate, 0.5))
    #n_hidden_units = int(n_hidden_units_row)
    #n_hidden_units = max(1, min(n_hidden_units, 512)) example uper bound, according to the data

    kf = KFold(n_splits=5, shuffle=True, random_state=42) #n_splits can be 5 or 10. in our case, we use 10-fold
    mse_scores = []

    for train_index, val_index in kf.split(normalized_features):
        X_train, X_val = normalized_features[train_index], normalized_features[val_index]
        y_train, y_val = target[train_index], target[val_index]

        model = create_grn_vsn_model(input_shape=(normalized_features.shape[1],), dropout_rate=dropout_rate)#it n_hidden_units need to put, it should be added here

        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

        loss, _ = model.evaluate(X_val, y_val, verbose=0)
        mse_scores.append(loss)

    return -np.mean(mse_scores)

optimizer = BayesianOptimization(f=train_model, pbounds={'lr': (0.00001, 0.001), 'dropout_rate': (0.1, 0.5)}, verbose=2) #n_hidden_units can be added here if it needed.
optimizer.maximize(init_points=5, n_iter=10) #n_iter can be added for better results if it needed.

# 8. Training and test
best_params = optimizer.max['params']
kf = KFold(n_splits=5, shuffle=True, random_state=42) #here n_splits can be 5 or 10. It is the x-fold that you need for your task.

final_mse_scores = []
for train_index, val_index in kf.split(normalized_features):
    X_train, X_val = normalized_features[train_index], normalized_features[val_index]
    y_train, y_val = target[train_index], target[val_index]

    final_model = create_grn_vsn_model(input_shape=(normalized_features.shape[1],), dropout_rate=best_params['dropout_rate'])
    final_model.compile(optimizer=Adam(learning_rate=best_params['lr']), loss='mean_squared_error',
                        metrics=[tf.keras.metrics.MeanSquaredError(name='mse_col1'), tf.keras.metrics.MeanSquaredError(name='mse_col2')])

    early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True) #patience can also be tuned accordingly.
    final_model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping]) #epochs abd the barch_size can be tuned also.

    loss, mse_col1, mse_col2 = final_model.evaluate(X_val, y_val)
    final_mse_scores.append(loss)
    print(f"Fold test MSE: {loss}\nTrait 1 test MSE: {mse_col1}\nTrait 2 test MSE: {mse_col2}")

print(f"Average Test MSE across 5 folds: {np.mean(final_mse_scores)}")